In [9]:
import csv
from datetime import timedelta
import gurobipy as gp 
from gurobipy import GRB
import networkx as nx
import random
import matplotlib.pyplot as plt
import inspect
from collections import defaultdict

from models import simple_mpc, mpc_duration_constr, lazy, column_generation
from helper import Service, hhmm2mins, mins2hhmm, fetch_data, draw_graph_with_edges, node_legal, no_overlap, create_duty_graph, extract_nodes, generate_paths, roster_statistics, solution_verify, solving_RMLP_art_vars_final

In [10]:
services, service_dict = fetch_data('./StepBackServices.csv')
graph = create_duty_graph(services)

In [11]:
init_duties, duty_count = mpc_duration_constr(graph, service_dict, time_limit = 30, show_logs = True, show_duties = False, show_roster_stats = True)

Set parameter TimeLimit to value 30
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 4600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  30

Optimize a model with 104364 rows, 68020 columns and 335422 nonzeros
Model fingerprint: 0x4522e447
Variable types: 34478 continuous, 33542 integer (33542 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 4061 rows and 1230 columns
Presolve time: 1.29s
Presolved: 100303 rows, 66790 columns, 373298 nonzeros
Variable types: 33424 continuous, 33366 integer (33366 binary)
Found heuristic solution: objective 900.0000000
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved 

In [12]:
selected_duties, dual_values, selected_duties_vars = solving_RMLP_art_vars_final(services, init_duties)
# selected_duties

Optimal solution found
Positive Duties:  146
x0 1.0
x1 1.0
x2 1.0
x3 1.0
x4 1.0
x5 1.0
x6 1.0
x7 1.0
x8 1.0
x9 1.0
x10 1.0
x11 1.0
x12 1.0
x13 1.0
x14 1.0
x15 1.0
x16 1.0
x17 1.0
x18 1.0
x19 1.0
x20 1.0
x21 1.0
x22 1.0
x23 1.0
x24 1.0
x25 1.0
x26 1.0
x27 1.0
x28 1.0
x29 1.0
x30 1.0
x31 1.0
x32 1.0
x33 1.0
x34 1.0
x35 1.0
x36 1.0
x37 1.0
x38 1.0
x39 1.0
x40 1.0
x41 1.0
x42 1.0
x43 1.0
x44 1.0
x45 1.0
x46 1.0
x47 1.0
x48 1.0
x49 1.0
x50 1.0
x51 1.0
x52 1.0
x53 1.0
x54 1.0
x55 1.0
x56 1.0
x57 1.0
x58 1.0
x59 1.0
x60 1.0
x61 1.0
x62 1.0
x63 1.0
x64 1.0
x65 1.0
x66 1.0
x67 1.0
x68 1.0
x69 1.0
x70 1.0
x71 1.0
x72 1.0
x73 1.0
x74 1.0
x75 1.0
x76 1.0
x77 1.0
x78 1.0
x79 1.0
x80 1.0
x81 1.0
x82 1.0
x83 1.0
x84 1.0
x85 1.0
x86 1.0
x87 1.0
x88 1.0
x89 1.0
x90 1.0
x91 1.0
x92 1.0
x93 1.0
x94 1.0
x95 1.0
x96 1.0
x97 1.0
x98 1.0
x99 1.0
x100 1.0
x101 1.0
x102 1.0
x103 1.0
x104 1.0
x105 1.0
x106 1.0
x107 1.0
x108 1.0
x109 1.0
x110 1.0
x111 1.0
x112 1.0
x113 1.0
x114 1.0
x115 1.0
x116 1.0
x117 1.0
x11

In [ ]:
def generate_new_column(graph, service_dict, current_duties, dual_values, method = "topological sort"):
    new_duties = []
    reduced_costs = []
    # Update edge weights based on dual values
    for u, v in graph.edges():
        graph[u][v]['weight'] = int(dual_values[u])* service_dict[u] # Adjust edge weight by dual value